In [11]:
# --- Kaggle Cell 1: Set dataset name and copy to /kaggle/working ---
DATASET_NAME="challenge-ai"  # CHANGE this to match your dataset folder name in /kaggle/input/

!rm -rf /kaggle/working/proj
!mkdir -p /kaggle/working/proj/data

# Copy train and test folders into writeable /kaggle/working
!cp -r /kaggle/input/{DATASET_NAME}/train /kaggle/working/proj/data/
!cp -r /kaggle/input/{DATASET_NAME}/test  /kaggle/working/proj/data/

# (Optional) Copy sample_submission.csv if it exists
!cp /kaggle/input/{DATASET_NAME}/sample_submission.csv /kaggle/working/proj/ 2>/dev/null || true


In [13]:
import os, random, shutil, glob
random.seed(42)

BASE = "/kaggle/working/proj/data"
ti, tl = f"{BASE}/train/images", f"{BASE}/train/labels"
vi, vl = f"{BASE}/val/images",   f"{BASE}/val/labels"
os.makedirs(vi, exist_ok=True); os.makedirs(vl, exist_ok=True)

imgs = sorted(glob.glob(os.path.join(ti, "*")))
val = set(random.sample(imgs, max(1, int(0.12*len(imgs)))))

stem = lambda p: os.path.splitext(os.path.basename(p))[0]
for ip in imgs:
    lp = os.path.join(tl, stem(ip)+".txt")
    if ip in val:
        shutil.move(ip, os.path.join(vi, os.path.basename(ip)))
        if os.path.exists(lp):
            shutil.move(lp, os.path.join(vl, os.path.basename(lp)))

print("Train images:", len(glob.glob(ti+"/*")))
print("Val images:", len(glob.glob(vi+"/*")))


Train images: 136
Val images: 38


In [14]:
yaml_text = """path: /kaggle/working/proj/data
train: train/images
val: val/images
test: test/images
names:
  0: bicycle
  1: bus
  2: car
  3: cng
  4: leguna
  5: manual-van
  6: motor
  7: others
  8: pedestrian
  9: rickshaw
  10: truck
"""
open("/kaggle/working/proj/data.yaml","w").write(yaml_text)
print(open("/kaggle/working/proj/data.yaml").read())


path: /kaggle/working/proj/data
train: train/images
val: val/images
test: test/images
names:
  0: bicycle
  1: bus
  2: car
  3: cng
  4: leguna
  5: manual-van
  6: motor
  7: others
  8: pedestrian
  9: rickshaw
  10: truck



In [15]:
# Install required packages
!pip install -q ultralytics ensemble-boxes

# Test that ultralytics imports correctly
from ultralytics import YOLO
print("Ultralytics imported OK")


Ultralytics imported OK


In [16]:
import sys, subprocess, importlib, platform, os

def sh(cmd):
    """Run a shell command and stream output (Python-only cell)."""
    print(f"$ {cmd}")
    return subprocess.run(cmd, shell=True, check=False)

print("=== GPU (nvidia-smi) ===")
sh("nvidia-smi")

print("\n=== Python & Torch status ===")
try:
    import torch
    print("Python:", platform.python_version())
    print("Torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    print("CUDA device count:", torch.cuda.device_count() if torch.cuda.is_available() else 0)
    had_torch = True
except Exception as e:
    print("Torch import failed:", e)
    had_torch = False

need_cuda = (not had_torch) or (had_torch and not torch.cuda.is_available())

if need_cuda:
    print("\n--> Installing CUDA-enabled PyTorch (cu124)...")
    # Uninstall any CPU builds
    sh(f"{sys.executable} -m pip uninstall -y torch torchvision torchaudio")
    # Install CUDA wheels
    sh(f"{sys.executable} -m pip install -q --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio")

    # IMPORTANT: If Torch was already imported, some environments need a restart to load new binaries.
    if had_torch:
        print("\nNOTE: Torch was already imported earlier in this session.")
        print("If the re-check below still shows CUDA = False, go to Runtime → Restart session, then run this cell again.")
    # Re-import / re-check
    try:
        if "torch" in sys.modules:
            del sys.modules["torch"]
        import torch as torch2
        print("\n=== Re-check Torch after install ===")
        print("Torch:", torch2.__version__)
        print("CUDA available:", torch2.cuda.is_available())
        print("CUDA device count:", torch2.cuda.device_count() if torch2.cuda.is_available() else 0)
    except Exception as e:
        print("Re-import error:", e)
else:
    print("\nCUDA is already available ✓")


=== GPU (nvidia-smi) ===
$ nvidia-smi
Sat Aug  9 09:09:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             33W /   70W |     703MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------

In [25]:
import subprocess, sys
cmd = (
    "yolo detect train "
    "data=/kaggle/working/proj/data.yaml "
    "model=yolov8m.pt "
    "imgsz=1536 "
    "batch=4 "
    "epochs=50 "
    "optimizer=AdamW lr0=0.001 cos_lr=True "
    "mosaic=1.0 mixup=0.15 "
    "hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 "
    "translate=0.1 scale=0.5 fliplr=0.5 "
    "cache=True patience=10 "
    "workers=2 "
    "device=0 "
    "project=/kaggle/working/proj/runs"
)
print("$", cmd)
subprocess.run(cmd, shell=True, check=False)

$ yolo detect train data=/kaggle/working/proj/data.yaml model=yolov8m.pt imgsz=1536 batch=4 epochs=50 optimizer=AdamW lr0=0.001 cos_lr=True mosaic=1.0 mixup=0.15 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 translate=0.1 scale=0.5 fliplr=0.5 cache=True patience=10 workers=2 device=0 project=/kaggle/working/proj/runs
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/proj/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1536, int8=False, iou=0.7, keras=False, kobj=1.0, line_wid

train: Scanning /kaggle/working/proj/data/train/labels.cache... 136 images, 0 backgrounds, 0 corrupt: 100%|██████████| 136/136 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.5GB RAM): 100%|██████████| 136/136 [00:02<00:00, 57.27it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1856.5±1319.6 MB/s, size: 772.6 KB)


...

      50/50      9.31G      1.057     0.6535      1.077        193       1536: 100%|██████████| 34/34 [00:28<00:00,  1.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  2.84it/s]


                   all         38       1163      0.743      0.606      0.679      0.394

50 epochs completed in 0.458 hours.
Optimizer stripped from /kaggle/working/proj/runs/train/weights/last.pt, 52.2MB
Optimizer stripped from /kaggle/working/proj/runs/train/weights/best.pt, 52.2MB

Validating /kaggle/working/proj/runs/train/weights/best.pt...
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.73it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         38       1163      0.697       0.65      0.679      0.394
               bicycle         15         17      0.845      0.588      0.716      0.402
                   bus          7         22      0.614      0.773      0.755       0.51
                   car         34        168       0.68      0.804      0.802      0.469
                   cng         26         80      0.683      0.838      0.835      0.536
                leguna          2          6      0.787      0.623      0.712      0.407
            manual-van         17         32      0.827      0.594      0.717      0.432
                 motor         28        100      0.658       0.55      0.582       0.29
                others          8         12      0.847      0.464      0.606       0.41
            pedestrian         36        289      0.556      0.626      0.569      0.253
              rickshaw         36        423      0.719      0.794      0.804      0.454
                 truc

CompletedProcess(args='yolo detect train data=/kaggle/working/proj/data.yaml model=yolov8m.pt imgsz=1536 batch=4 epochs=50 optimizer=AdamW lr0=0.001 cos_lr=True mosaic=1.0 mixup=0.15 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 translate=0.1 scale=0.5 fliplr=0.5 cache=True patience=10 workers=2 device=0 project=/kaggle/working/proj/runs', returncode=0)

In [26]:
!yolo detect predict \
  model=/kaggle/working/proj/runs/train/weights/best.pt \
  source=/kaggle/working/proj/data/test/images \
  imgsz=1280 conf=0.05 iou=0.45 \
  save_txt=True save_conf=True max_det=2000 \
  project=/kaggle/working/proj/runs --name=predict


WARNING ⚠️ argument '--name=predict' does not require leading dashes '--', updating to 'name=predict'.
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs

image 1/57 /kaggle/working/proj/data/test/images/DJI_0017_000057_jpg.rf.a2ce1add5011cc942c1c1501c65e5eff.jpg: 704x1280 8 cars, 1 cng, 4 motors, 1 pedestrian, 10 rickshaws, 71.1ms
image 2/57 /kaggle/working/proj/data/test/images/DJI_0018_000001_jpg.rf.e11080fe695e8a6f1c4bee916401199a.jpg: 704x1280 6 cars, 2 manual-vans, 4 pedestrians, 16 rickshaws, 52.7ms
image 3/57 /kaggle/working/proj/data/test/images/DJI_0018_000003_jpg.rf.4d6950f51b4bda0e8b884bf342fb6466.jpg: 704x1280 6 cars, 1 cng, 2 manual-vans, 2 motors, 1 pedestrian, 13 rickshaws, 52.7ms
image 4/57 /kaggle/working/proj/data/test/images/DJI_0019_000001_jpg.rf.e0d9ea54ee3f8606ae2897d2398a9a1b.jpg: 704x1280 4 cars, 1 cng, 3 pedestrians, 8 rickshaws, 52.7ms
image 5/57 

In [27]:
import os, glob, pandas as pd
from PIL import Image

TEST_DIR   = "/kaggle/working/proj/data/test/images"
LABELS_DIR = "/kaggle/working/proj/runs/predict/labels"
OUT_CSV    = "/kaggle/working/submission_okay.csv"

# image_id (no extension) -> (w,h)
sizes = {}
for p in glob.glob(os.path.join(TEST_DIR, "*")):
    name = os.path.splitext(os.path.basename(p))[0]
    with Image.open(p) as im:
        sizes[name] = im.size  # (w,h)

rows, seen = [], set()
for t in sorted(glob.glob(os.path.join(LABELS_DIR, "*.txt"))):
    image_id = os.path.splitext(os.path.basename(t))[0]
    w, h = sizes[image_id]
    parts_out = []
    with open(t) as f:
        for line in f:
            vals = line.strip().split()
            if len(vals) not in (5,6): 
                continue
            cls = int(float(vals[0]))
            x, y, bw, bh = map(float, vals[1:5])  # normalized
            conf = float(vals[5]) if len(vals) == 6 else 0.5

            x_min = (x - bw/2) * w
            y_min = (y - bh/2) * h
            x_max = (x + bw/2) * w
            y_max = (y + bh/2) * h

            parts_out += [
                str(cls), f"{conf:.6f}",
                str(int(max(0, x_min))), str(int(max(0, y_min))),
                str(int(min(w-1, x_max))), str(int(min(h-1, y_max))),
            ]
    rows.append({"image_id": image_id, "PredictionString": " ".join(parts_out)})
    seen.add(image_id)

# empty PredictionString for images with no detections
for name in sizes.keys():
    if name not in seen:
        rows.append({"image_id": name, "PredictionString": ""})

pd.DataFrame(rows).sort_values("image_id").to_csv(OUT_CSV, index=False)
print("Wrote", OUT_CSV)


Wrote /kaggle/working/submission_okay.csv
